In [ ]:
import time
import smtplib
import requests
import pywhatkit
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# User configurations
FLIPKART_EMAIL = 'your_flipkart_email'
FLIPKART_PASSWORD = 'your_flipkart_password'
PRODUCT_URL = 'https://www.flipkart.com/product-url'
DESIRED_PRICE = 1000  # Set your desired price
CHECK_INTERVAL = 300  # seconds

# Notification configurations
EMAIL_SENDER = 'your_email@gmail.com'
EMAIL_PASSWORD = 'your_email_password'
EMAIL_RECEIVER = 'receiver_email@gmail.com'
WHATSAPP_NUMBER = '+91XXXXXXXXXX'  # Include country code

def login_flipkart():
    # Initialize WebDriver
    driver = webdriver.Chrome()
    driver.get('https://www.flipkart.com/login')
    time.sleep(3)

    # Login process
    email_input = driver.find_element(By.CLASS_NAME, '_2IX_2- VJZDxU')
    password_input = driver.find_element(By.CLASS_NAME, '_2IX_2- _3mctL4')
    email_input.send_keys(FLIPKART_EMAIL)
    password_input.send_keys(FLIPKART_PASSWORD)
    password_input.send_keys(Keys.RETURN)
    time.sleep(5)
    return driver

def get_product_price():
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(PRODUCT_URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Flipkart's current price element
    price_element = soup.find('div', {'class': '_30jeq3 _16Jk6d'})
    if price_element:
        price_text = price_element.text.replace('₹','').replace(',','')
        return float(price_text)
    return None

def send_email(price):
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(EMAIL_SENDER, EMAIL_PASSWORD)
        subject = 'Price Drop Alert'
        body = f'The product has dropped to {price}.\nCheck it here: {PRODUCT_URL}'
        message = f'Subject: {subject}\n\n{body}'
        server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, message)

def send_whatsapp():
    # Send WhatsApp message using pywhatkit
    message = f"Price alert! Check the product: {PRODUCT_URL}"
    pywhatkit.sendwhatmsg_instantly(WHATSAPP_NUMBER, message, 10)

def main():
    # Optional: login to Flipkart to maintain session for scraping
    driver = login_flipkart()
    driver.quit()
    while True:
        price = get_product_price()
        if price and price <= DESIRED_PRICE:
            send_email(price)
            send_whatsapp()
            print(f"Notification sent! Price: ₹{price}")
            break
        else:
            print(f"Current Price: ₹{price}, sleeping for {CHECK_INTERVAL} seconds")
        time.sleep(CHECK_INTERVAL)

if __name__ == "__main__":
    main()